## Análise de dados da fila da creche no município de São Paulo

Obtendo os dados das matrículas na rede municipal de educação, obtidos [aqui](http://dados.prefeitura.sp.gov.br/dataset/demanda-por-vagas-educacao-infantil-e-eja):

In [1]:
import pandas as pd

data_creche = pd.read_csv('dados/evolucaodemanda.csv', sep = ';')

Temos que filtrar apenas as colunas que necessárias. Além disso, é preciso limpar os dados, pois algumas linhas possuem os números com um ponto (.) para indicar o milhar. Por conveniência, o ano de referência será deixado em uma coluna a parte. 

In [2]:
data_creche = data_creche[['DISTRITO', 'MAT_CRECHE', 'DEM_CRECHE', 'MES_ANO_REF']]

for i in data_creche.index:
    if not data_creche.at[i, 'MAT_CRECHE'].is_integer():
        data_creche.at[i, 'MAT_CRECHE'] *= 1000
    if not data_creche.at[i, 'DEM_CRECHE'].is_integer():
        data_creche.at[i, 'DEM_CRECHE'] *= 1000
    data_creche.at[i, 'ANO'] = int(data_creche.at[i, 'MES_ANO_REF'][4:]) + 2000

Agora, vamos obter os dados populacionais por distrito, com os dados da [SEADE](http://produtos.seade.gov.br/produtos/projpop/):

In [10]:
data_pop_2011 = pd.read_csv('dados/populacao/DIS_FXE_DIS_2011.csv', sep = ';', encoding = "ISO-8859-1")
data_pop_2012 = pd.read_csv('dados/populacao/DIS_FXE_DIS_2012.csv', sep = ';', encoding = "ISO-8859-1")
data_pop_2013 = pd.read_csv('dados/populacao/DIS_FXE_DIS_2013.csv', sep = ';', encoding = "ISO-8859-1")
data_pop_2014 = pd.read_csv('dados/populacao/DIS_FXE_DIS_2014.csv', sep = ';', encoding = "ISO-8859-1")
data_pop_2015 = pd.read_csv('dados/populacao/DIS_FXE_DIS_2015.csv', sep = ';', encoding = "ISO-8859-1")
data_pop_2016 = pd.read_csv('dados/populacao/DIS_FXE_DIS_2016.csv', sep = ';', encoding = "ISO-8859-1")
data_pop_2017 = pd.read_csv('dados/populacao/DIS_FXE_DIS_2017.csv', sep = ';', encoding = "ISO-8859-1")

Vamos filtrar apenas as colunas necessárias aqui também. Juntamente com isso, é preciso tratar os dados da população, assim como os dados da creche. A diferença é que aqui teremos que excluir linhas. Os arquivos fornecidos têm as linhas no seguinte padrão:

   - ID do distrito, Nome do Distrito, Ano do dado, Seleção feita, População (linha da faixa etária);
   - ID do distrito, Nome do Distrito, Ano do dado, Seleção feita, População (linha da seleção, no caso, igual à anterior);
   - ID do distrito, Nome do Distrito, Ano do dado, Seleção feita, População (linha da população geral do distrito).
   
Precisamos apenas das linhas que correspondem ao primeiro padrão. Além disso, os dados que representam a população serão limpos pois alguns possuem um ponto (.), que representam o milhar. Para que os dados populacionais fiquem mais próximos aos da creche, vamos alterar também o nome do distrito, deixando-o sem acentos e com caixa alta.

In [11]:
def arrumaDados(dado):
    
    # Filtra as colunas
    dado = dado[['Nome Localidade', 'Ano', 'Total']]
    
    # Pega os índices para serem excluídos
    indices = list()
    for i in dado.index:
        if i % 3 != 0:
            indices.append(i)
    dado = dado.drop(dado.index[indices])
    
    # Tira os acentos dos nomes dos distritos
    dado['Nome Localidade'] = dado['Nome Localidade'].str.normalize('NFKD')
    dado['Nome Localidade'] = dado['Nome Localidade'].str.encode('ascii', errors='ignore').str.decode('utf-8')
    
    # Deixa o nome de cada distrito em caixa alta e tira a pontuação dos valores no total de população
    for i in dado.index:
        dado.at[i, 'Nome Localidade'] = dado.at[i, 'Nome Localidade'].upper()
        if not dado.at[i, 'Total'].is_integer():
            dado.at[i, 'Total'] *= 1000
    return dado

data_pop_2011 = arrumaDados(data_pop_2011)
data_pop_2012 = arrumaDados(data_pop_2012)
data_pop_2013 = arrumaDados(data_pop_2013)
data_pop_2014 = arrumaDados(data_pop_2014)
data_pop_2015 = arrumaDados(data_pop_2015)
data_pop_2016 = arrumaDados(data_pop_2016)
data_pop_2017 = arrumaDados(data_pop_2017)

Vamos juntar os dados de todos os anos para termos apenas um conjunto de dados referente à população. Além disso, as colunas terão os mesmos nomes nas nossas duas bases de dados (creche e população).

In [16]:
data_pop = pd.concat([data_pop_2011, data_pop_2012, data_pop_2013, data_pop_2014, data_pop_2015, data_pop_2016, data_pop_2017])
data_pop.columns = ['DISTRITO', 'ANO', 'TOTAL']

Precisamos juntar os dados, para que a análise seja feita de maneira mais eficiente. Teremos um dicionário com os seguintes valores:

- Nome do distrito (em caixa alta e sem acentos);
- Número de matriculas em creches;
- Demanda de vagas em creches;
- População do distrito na faixa etária 00 a 03 anos;
- Período de referência (mmm-aa).
- População total naquele ano.

In [18]:
data_creche_pop = data_creche.merge(data_pop, on = ['DISTRITO', 'ANO'])